#### Sistema de analisis automatico de callos vegetales en placas de Petri.

**Practica C: Deteccion de patrones y segmentacion**

In [ ]:
pip install numpy scipy opencv-python matplotlib pandas pillow scikit-image


In [14]:
import numpy as np
import cv2
import pandas as pd
from pathlib import Path
from datetime import datetime
import json
from base64 import b64decode
from io import BytesIO
from PIL import Image
import scipy.signal
import scipy.ndimage
import matplotlib.pyplot as plt
from tqdm import tqdm

**PATH a la carpeta con las imagenes**

In [4]:
IMAGES_PATH = "C:/Users/Usuario/Documents/Bioimagenes/Trabajo entregable bioimagen/CallosVegetales"

In [ ]:
class AnalizadorCallos:
    def __init__(self, carpeta_imagenes, carpeta_salida="resultados"):
        self.carpeta_imagenes = Path(carpeta_imagenes)
        self.carpeta_salida = Path(carpeta_salida)
        self.carpeta_salida.mkdir(exist_ok=True)
        (self.carpeta_salida / "log_sigma").mkdir(exist_ok=True)
        (self.carpeta_salida / "visualizaciones").mkdir(exist_ok=True)

        self.area_minima_callos = 100
        self.area_maxima_callos = 500000
        self.area_minima_semillas = 10
        self.area_maxima_semillas = 1000
        self.mm_por_pixel = 0.1

        self.kernel_laplaciano = np.asarray(
            [[0, 1, 0],
             [1, -4, 1],
             [0, 1, 0]], dtype=float
        )

        self.escalas = [0.5, 1, 2, 3, 5]

    def cargar_imagen(self, ruta_imagen):
        imagen = cv2.imread(str(ruta_imagen), cv2.IMREAD_UNCHANGED)
        if imagen is None:
            raise ValueError("No se pudo cargar: {}".format(ruta_imagen))

        if len(imagen.shape) == 3 and imagen.shape[2] == 4:
            return imagen[:, :, :3], imagen[:, :, 3]

        return imagen, None

    def cargar_objetos_json(self, ruta_json, forma_imagen):
        if not ruta_json.exists():
            return {"callos": [], "semillas": []}, {"callos": 0, "semillas": 0}

        with open(ruta_json, "r", encoding="utf-8") as archivo:
            datos = json.load(archivo)

        mascaras_callos = []
        mascaras_semillas = []
        num_callos = 0
        num_semillas = 0

        for forma in datos.get("shapes", []):
            etiqueta = forma.get("label", "")
            tipo_forma = forma.get("shape_type", "")
            
            if tipo_forma == "mask":
                mascara_b64 = forma.get("mask")
                puntos = forma.get("points", [])

                if mascara_b64 and len(puntos) == 2:
                    mascara_bytes = b64decode(mascara_b64)
                    mascara_img = Image.open(BytesIO(mascara_bytes)).convert("L")
                    mascara_array = np.array(mascara_img)

                    mascara_binaria = (mascara_array > 0).astype(np.uint8) * 255

                    x1, y1 = int(puntos[0][0]), int(puntos[0][1])
                    x2, y2 = int(puntos[1][0]), int(puntos[1][1])

                    x1, y1 = max(0, x1), max(0, y1)
                    x2 = min(forma_imagen[1], x2)
                    y2 = min(forma_imagen[0], y2)

                    mascara_completa = np.zeros(forma_imagen[:2], dtype=np.uint8)
                    
                    h, w = y2 - y1, x2 - x1
                    if mascara_binaria.shape != (h, w):
                        mascara_binaria = cv2.resize(
                            mascara_binaria, (w, h), 
                            interpolation=cv2.INTER_NEAREST)
                    
                    if h > 0 and w > 0:
                        mascara_completa[y1:y2, x1:x2] = mascara_binaria
                        
                        if etiqueta == "callo":
                            mascaras_callos.append(mascara_completa)
                            num_callos += 1
                        elif etiqueta == "semilla":
                            mascaras_semillas.append(mascara_completa)
                            num_semillas += 1

        return {"callos": mascaras_callos, "semillas": mascaras_semillas}, {
            "callos": num_callos, "semillas": num_semillas}

    def analizar_mascara_individual(self, mascara, imagen_original, 
                                   canal_alpha=None, idx=1, tipo="callo"):
        contornos, _ = cv2.findContours(
            mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contornos) == 0:
            return None
        
        contorno = max(contornos, key=cv2.contourArea)
        area_px = cv2.contourArea(contorno)
        
        if tipo == "callo":
            if (area_px < self.area_minima_callos or 
                area_px > self.area_maxima_callos):
                return None
        elif tipo == "semilla":
            if (area_px < self.area_minima_semillas or 
                area_px > self.area_maxima_semillas):
                return None
        
        momentos = cv2.moments(contorno)
        if momentos["m00"] != 0:
            cx = int(momentos["m10"] / momentos["m00"])
            cy = int(momentos["m01"] / momentos["m00"])
        else:
            cx, cy = 0, 0

        perimetro = cv2.arcLength(contorno, True)
        circularidad = (4 * np.pi * area_px / (perimetro ** 2) 
                       if perimetro > 0 else 0)
        area_mm2 = area_px * (self.mm_por_pixel ** 2)

        opacidad_media = None
        if canal_alpha is not None:
            mascara_contorno = np.zeros_like(canal_alpha)
            cv2.drawContours(mascara_contorno, [contorno], -1, 255, -1)
            opacidad_media = np.mean(canal_alpha[mascara_contorno > 0])

        return {
            "callo_id": idx,
            "tipo": tipo,
            "area_pixeles": area_px,
            "area_mm2": area_mm2,
            "centroide_x": cx,
            "centroide_y": cy,
            "perimetro": perimetro,
            "circularidad": circularidad,
            "opacidad_media": opacidad_media
        }

    def preprocesar_imagen(self, imagen, canal_alpha=None):
        imagen_difuminada = cv2.GaussianBlur(imagen, (5, 5), 0)
        hsv = cv2.cvtColor(imagen_difuminada, cv2.COLOR_BGR2HSV)
        gris = cv2.cvtColor(imagen_difuminada, cv2.COLOR_BGR2GRAY)

        mascara_alpha = None
        if canal_alpha is not None:
            _, mascara_alpha = cv2.threshold(
                canal_alpha, 10, 255, cv2.THRESH_BINARY)

        return hsv, gris, mascara_alpha

    def segmentar_callo(self, imagen_hsv, mascara_alpha=None, usar_alpha=True):
        verde_bajo = np.array([25, 40, 40])
        verde_alto = np.array([85, 255, 255])
        mascara_color = cv2.inRange(imagen_hsv, verde_bajo, verde_alto)
        _, s, _ = cv2.split(imagen_hsv)
        _, mascara_saturacion = cv2.threshold(s, 30, 255, cv2.THRESH_BINARY)
        mascara_combinada = cv2.bitwise_and(mascara_color, mascara_saturacion)

        if mascara_alpha is not None and usar_alpha:
            mascara_combinada = cv2.bitwise_and(mascara_combinada, mascara_alpha)

        return mascara_combinada

    def postprocesar_mascara(self, mascara):
        kernel_abrir = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        kernel_cerrar = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
        mascara = cv2.morphologyEx(mascara, cv2.MORPH_OPEN, 
                                   kernel_abrir, iterations=2)
        mascara = cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, 
                                   kernel_cerrar, iterations=3)
        return mascara

    def aplicar_filtro_log(self, imagen, sigma):
        imagen_gaussiana = scipy.ndimage.gaussian_filter(imagen, sigma=sigma)
        imagen_log = scipy.signal.convolve2d(
            imagen_gaussiana, self.kernel_laplaciano, 
            mode="same", boundary="wrap")
        return imagen_log

    def procesar_log_multiescala(self, imagen_gris, nombre_imagen):
        resultados_log = []
        for sigma in self.escalas:
            imagen_log = self.aplicar_filtro_log(imagen_gris, sigma)
            resultados_log.append(imagen_log)
            
            figura = plt.figure(figsize=(8, 6))
            plt.imshow(imagen_log, cmap="gray")
            plt.title("LoG - Escala sigma={}".format(sigma))
            plt.colorbar(label="Intensidad")
            plt.axis("off")
            
            ruta_salida = (self.carpeta_salida / "log_sigma" / 
                          "log_{}_{}.png".format(nombre_imagen, sigma))
            plt.savefig(ruta_salida, dpi=150, bbox_inches="tight")
            plt.close()
        return resultados_log

    def analizar_blobs(self, mascara, imagen_original, 
                      canal_alpha=None, tipo="callo"):
        if tipo == "callo":
            area_minima = self.area_minima_callos
            area_maxima = self.area_maxima_callos
        else:
            area_minima = self.area_minima_semillas
            area_maxima = self.area_maxima_semillas
            
        contornos, _ = cv2.findContours(
            mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        resultados = []
        imagen_anotada = imagen_original.copy()
        idx = 1
        
        for contorno in contornos:
            area_px = cv2.contourArea(contorno)
            if area_minima < area_px < area_maxima:
                momentos = cv2.moments(contorno)
                if momentos["m00"] != 0:
                    cx = int(momentos["m10"] / momentos["m00"])
                    cy = int(momentos["m01"] / momentos["m00"])
                else:
                    cx, cy = 0, 0

                perimetro = cv2.arcLength(contorno, True)
                circularidad = (4 * np.pi * area_px / (perimetro ** 2) 
                              if perimetro > 0 else 0)
                area_mm2 = area_px * (self.mm_por_pixel ** 2)

                opacidad_media = None
                if canal_alpha is not None:
                    mascara_contorno = np.zeros_like(canal_alpha)
                    cv2.drawContours(mascara_contorno, [contorno], -1, 255, -1)
                    opacidad_media = np.mean(canal_alpha[mascara_contorno > 0])

                resultados.append({
                    "callo_id": idx,
                    "tipo": tipo,
                    "area_pixeles": area_px,
                    "area_mm2": area_mm2,
                    "centroide_x": cx,
                    "centroide_y": cy,
                    "perimetro": perimetro,
                    "circularidad": circularidad,
                    "opacidad_media": opacidad_media
                })

                if tipo == "callo":
                    color = (0, 255, 0)
                else:
                    color = (255, 0, 255)
                
                cv2.drawContours(imagen_anotada, [contorno], -1, color, 2)
                cv2.circle(imagen_anotada, (cx, cy), 5, (255, 0, 0), -1)
                cv2.putText(imagen_anotada, "#{}".format(idx), (cx + 10, cy), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                idx += 1
                
        return resultados, imagen_anotada

    def guardar_visualizacion(self, nombre_imagen, original, mascara_auto, 
                            objetos_json, anotada_auto, anotada_json, 
                            canal_alpha=None):
        n_filas = 3
        n_columnas = 3
        
        figura = plt.figure(figsize=(6 * n_columnas, 5 * n_filas))
        
        idx = 1
        plt.subplot(n_filas, n_columnas, idx)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title("Imagen Original", fontsize=12)
        plt.axis("off")
        idx += 1
        
        plt.subplot(n_filas, n_columnas, idx)
        plt.imshow(mascara_auto, cmap="gray")
        plt.title("Mascara Automatica", fontsize=12)
        plt.axis("off")
        idx += 1
        
        plt.subplot(n_filas, n_columnas, idx)
        plt.imshow(cv2.cvtColor(anotada_auto, cv2.COLOR_BGR2RGB))
        plt.title("Deteccion Automatica", fontsize=12)
        plt.axis("off")
        
        idx = 4
        plt.subplot(n_filas, n_columnas, idx)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title("Original", fontsize=12)
        plt.axis("off")
        idx += 1
        
        plt.subplot(n_filas, n_columnas, idx)
        if objetos_json["callos"]:
            mascara_combinada_callos = np.zeros_like(mascara_auto)
            for mascara in objetos_json["callos"]:
                mascara_combinada_callos = cv2.bitwise_or(
                    mascara_combinada_callos, mascara)
            plt.imshow(mascara_combinada_callos, cmap="gray")
            titulo = "Callos JSON ({} callos)".format(len(objetos_json["callos"]))
            plt.title(titulo, fontsize=12)
        else:
            plt.imshow(np.zeros_like(mascara_auto), cmap="gray")
            plt.title("Sin Callos JSON", fontsize=12)
        plt.axis("off")
        idx += 1
        
        plt.subplot(n_filas, n_columnas, idx)
        if anotada_json is not None:
            plt.imshow(cv2.cvtColor(anotada_json, cv2.COLOR_BGR2RGB))
            plt.title("Callos JSON Detectados", fontsize=12)
        else:
            plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
            plt.title("Sin Callos JSON", fontsize=12)
        plt.axis("off")
        
        idx = 7
        plt.subplot(n_filas, n_columnas, idx)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title("Original", fontsize=12)
        plt.axis("off")
        idx += 1
        
        plt.subplot(n_filas, n_columnas, idx)
        if objetos_json["semillas"]:
            mascara_combinada_semillas = np.zeros_like(mascara_auto)
            for mascara in objetos_json["semillas"]:
                mascara_combinada_semillas = cv2.bitwise_or(
                    mascara_combinada_semillas, mascara)
            plt.imshow(mascara_combinada_semillas, cmap="gray")
            titulo = "Semillas JSON ({} semillas)".format(len(objetos_json["semillas"]))
            plt.title(titulo, fontsize=12)
        else:
            plt.imshow(np.zeros_like(mascara_auto), cmap="gray")
            plt.title("Sin Semillas JSON", fontsize=12)
        plt.axis("off")
        idx += 1
        
        plt.subplot(n_filas, n_columnas, idx)
        if objetos_json["semillas"]:
            imagen_semillas = original.copy()
            semilla_idx = 1
            for mascara in objetos_json["semillas"]:
                contornos, _ = cv2.findContours(
                    mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contornos:
                    contorno = max(contornos, key=cv2.contourArea)
                    cv2.drawContours(imagen_semillas, [contorno], -1, 
                                    (255, 0, 255), 1)
                    
                    momentos = cv2.moments(contorno)
                    if momentos["m00"] != 0:
                        cx = int(momentos["m10"] / momentos["m00"])
                        cy = int(momentos["m01"] / momentos["m00"])
                        cv2.circle(imagen_semillas, (cx, cy), 2, (255, 0, 0), -1)
                        cv2.putText(imagen_semillas, "S{}".format(semilla_idx), 
                                   (cx + 5, cy), cv2.FONT_HERSHEY_SIMPLEX,
                                   0.4, (0, 0, 255), 1)
                        semilla_idx += 1
            plt.imshow(cv2.cvtColor(imagen_semillas, cv2.COLOR_BGR2RGB))
            plt.title("Semillas JSON Marcadas", fontsize=12)
        else:
            plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
            plt.title("Sin Semillas JSON", fontsize=12)
        plt.axis("off")
        
        plt.suptitle("Comparativa: {}".format(nombre_imagen), fontsize=14)
        plt.tight_layout()
        
        ruta_salida = (self.carpeta_salida / "visualizaciones" / 
                      "{}_comparacion.png".format(nombre_imagen))
        plt.savefig(ruta_salida, dpi=150, bbox_inches="tight")
        plt.close()

    def procesar_todas_imagenes(self, usar_alpha=True, usar_log=True, usar_json=True):
        archivos_imagenes = sorted(
            list(self.carpeta_imagenes.glob("*.png")) +
            list(self.carpeta_imagenes.glob("*.jpg")) +
            list(self.carpeta_imagenes.glob("*.tif"))
        )
        if not archivos_imagenes:
            mensaje = "No se encontraron imagenes en {}".format(self.carpeta_imagenes)
            raise ValueError(mensaje)

        todos_resultados = []
        info_comparacion = []

        print("\nPROCESANDO IMAGENES")
        print("_" * 40)
        print("Total de imagenes: {}".format(len(archivos_imagenes)))
        print("Carpeta: {}".format(self.carpeta_imagenes))
        print("Alpha: {}".format("Activado" if usar_alpha else "Desactivado"))
        print("Filtros LoG: {}".format("Activados" if usar_log else "Desactivados"))
        print("Anotaciones JSON: {}".format("Activadas" if usar_json else "Desactivadas"))
        print()

        for ruta_imagen in tqdm(archivos_imagenes, desc="Procesando", unit="img"):
            try:
                imagen, alpha = self.cargar_imagen(ruta_imagen)
                hsv, gris, mascara_alpha = self.preprocesar_imagen(imagen, alpha)

                if usar_log:
                    self.procesar_log_multiescala(gris, ruta_imagen.stem)

                mascara_auto = self.segmentar_callo(hsv, mascara_alpha, usar_alpha)
                mascara_auto_limpia = self.postprocesar_mascara(mascara_auto)
                resultados_auto, anotada_auto = self.analizar_blobs(
                    mascara_auto_limpia, imagen, alpha, tipo="callo")

                for resultado in resultados_auto:
                    resultado["imagen"] = ruta_imagen.name
                    resultado["fuente"] = "Automatico"
                    todos_resultados.append(resultado)

                objetos_json = {"callos": [], "semillas": []}
                resultados_json = []
                anotada_json = None

                if usar_json:
                    ruta_json = ruta_imagen.with_suffix(".json")
                    objetos_json, num_objetos = self.cargar_objetos_json(
                        ruta_json, imagen.shape)

                    info_img = {
                        "nombre_imagen": ruta_imagen.name,
                        "callos_auto": len(resultados_auto),
                        "callos_json": len(objetos_json["callos"]),
                        "centroides_auto": [],
                        "centroides_json": [],
                        "areas_auto": [],
                        "areas_json": []
                    }

                    if objetos_json["callos"]:
                        anotada_json = imagen.copy()
                        json_idx = 1
                        
                        for mascara in objetos_json["callos"]:
                            resultado = self.analizar_mascara_individual(
                                mascara, imagen, alpha, idx=json_idx, tipo="callo")
                            
                            if resultado:
                                resultado["imagen"] = ruta_imagen.name
                                resultado["fuente"] = "JSON"
                                todos_resultados.append(resultado)
                                resultados_json.append(resultado)
                                
                                info_img["centroides_json"].append((resultado["centroide_x"], resultado["centroide_y"]))
                                info_img["areas_json"].append(resultado["area_pixeles"])
                                
                                contornos, _ = cv2.findContours(
                                    mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                                if contornos:
                                    contorno = max(contornos, key=cv2.contourArea)
                                    cv2.drawContours(anotada_json, [contorno], -1, 
                                                    (0, 255, 0), 2)
                                    
                                    momentos = cv2.moments(contorno)
                                    if momentos["m00"] != 0:
                                        cx = int(momentos["m10"] / momentos["m00"])
                                        cy = int(momentos["m01"] / momentos["m00"])
                                        cv2.circle(anotada_json, (cx, cy), 3, 
                                                  (255, 0, 0), -1)
                                        cv2.putText(anotada_json, "#{}".format(json_idx), 
                                                   (cx + 8, cy), 
                                                   cv2.FONT_HERSHEY_SIMPLEX,
                                                   0.5, (0, 0, 255), 1)
                                    
                                    json_idx += 1

                    for resultado_auto in resultados_auto:
                        info_img["centroides_auto"].append((resultado_auto["centroide_x"], resultado_auto["centroide_y"]))
                        info_img["areas_auto"].append(resultado_auto["area_pixeles"])
                    
                    info_comparacion.append(info_img)

                    if objetos_json["semillas"]:
                        semilla_idx = 1
                        for mascara in objetos_json["semillas"]:
                            resultado = self.analizar_mascara_individual(
                                mascara, imagen, alpha, idx=semilla_idx, 
                                tipo="semilla")
                            
                            if resultado:
                                resultado["imagen"] = ruta_imagen.name
                                resultado["fuente"] = "JSON"
                                todos_resultados.append(resultado)
                                resultados_json.append(resultado)
                                semilla_idx += 1

                self.guardar_visualizacion(
                    ruta_imagen.stem, imagen, mascara_auto_limpia, objetos_json,
                    anotada_auto, anotada_json, alpha
                )

            except Exception as e:
                print("\nError procesando {}: {}".format(ruta_imagen.name, str(e)))
                continue

        return todos_resultados, info_comparacion

    def comparar_detecciones(self, info_comparacion, umbral_distancia=20):
        """
        Compara deteccion automatica vs anotaciones JSON.
        Calcula metricas de precision.
        """
        if not info_comparacion:
            return None
        
        total_verdaderos_positivos = 0
        total_falsos_positivos = 0
        total_falsos_negativos = 0
        
        resultados_por_imagen = []
        
        for info_img in info_comparacion:
            imagen = info_img["nombre_imagen"]
            centroides_json = info_img["centroides_json"]
            centroides_auto = info_img["centroides_auto"]
            areas_json = info_img["areas_json"]
            areas_auto = info_img["areas_auto"]
            
            verdaderos_positivos = 0
            falsos_positivos = 0
            falsos_negativos = 0
            
            json_detectado = [False] * len(centroides_json)
            auto_usado = [False] * len(centroides_auto)
            
            for i, (centroide_json, area_json) in enumerate(zip(centroides_json, areas_json)):
                detectado = False
                mejor_distancia = float('inf')
                mejor_j = -1
                
                for j, (centroide_auto, area_auto) in enumerate(zip(centroides_auto, areas_auto)):
                    if auto_usado[j]:
                        continue
                    
                    distancia = np.sqrt((centroide_json[0] - centroide_auto[0])**2 + 
                                       (centroide_json[1] - centroide_auto[1])**2)
                    
                    diferencia_area = abs(area_json - area_auto) / max(area_json, area_auto)
                    
                    if distancia < umbral_distancia and diferencia_area < 0.5:
                        if distancia < mejor_distancia:
                            mejor_distancia = distancia
                            mejor_j = j
                            detectado = True
                
                if detectado and mejor_j != -1:
                    verdaderos_positivos += 1
                    json_detectado[i] = True
                    auto_usado[mejor_j] = True
            
            falsos_positivos = sum(1 for usado in auto_usado if not usado)
            falsos_negativos = sum(1 for detectado in json_detectado if not detectado)
            
            precision = verdaderos_positivos / (verdaderos_positivos + falsos_positivos) if (verdaderos_positivos + falsos_positivos) > 0 else 0
            recall = verdaderos_positivos / (verdaderos_positivos + falsos_negativos) if (verdaderos_positivos + falsos_negativos) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            
            resultados_por_imagen.append({
                "imagen": imagen,
                "verdaderos_positivos": verdaderos_positivos,
                "falsos_positivos": falsos_positivos,
                "falsos_negativos": falsos_negativos,
                "precision": precision,
                "recall": recall,
                "f1": f1
            })
            
            total_verdaderos_positivos += verdaderos_positivos
            total_falsos_positivos += falsos_positivos
            total_falsos_negativos += falsos_negativos
        
        precision_global = total_verdaderos_positivos / (total_verdaderos_positivos + total_falsos_positivos) if (total_verdaderos_positivos + total_falsos_positivos) > 0 else 0
        recall_global = total_verdaderos_positivos / (total_verdaderos_positivos + total_falsos_negativos) if (total_verdaderos_positivos + total_falsos_negativos) > 0 else 0
        f1_global = 2 * precision_global * recall_global / (precision_global + recall_global) if (precision_global + recall_global) > 0 else 0
        
        return {
            "metricas_globales": {
                "verdaderos_positivos": total_verdaderos_positivos,
                "falsos_positivos": total_falsos_positivos,
                "falsos_negativos": total_falsos_negativos,
                "precision": precision_global,
                "recall": recall_global,
                "f1": f1_global
            },
            "metricas_por_imagen": resultados_por_imagen
        }

    def generar_reporte(self, resultados, info_comparacion=None):
        df = pd.DataFrame(resultados)
        
        if len(df) == 0:
            print("\nNo se encontraron objetos en las imagenes")
            return df, None

        df_callos = df[df["tipo"] == "callo"]
        df_semillas = df[df["tipo"] == "semilla"]
        
        df_callos_auto = df_callos[df_callos["fuente"] == "Automatico"]
        df_callos_json = df_callos[df_callos["fuente"] == "JSON"]
        df_semillas_json = df_semillas[df_semillas["fuente"] == "JSON"]

        total_objetos = len(df)
        total_callos = len(df_callos)
        total_semillas = len(df_semillas)
        
        callos_auto = len(df_callos_auto)
        callos_json = len(df_callos_json)
        semillas_json = len(df_semillas_json)
        
        area_total = df["area_mm2"].sum()
        area_callos = df_callos["area_mm2"].sum()
        area_semillas = df_semillas["area_mm2"].sum()
        
        area_callos_auto = df_callos_auto["area_mm2"].sum()
        area_callos_json = df_callos_json["area_mm2"].sum()
        area_semillas_json = df_semillas_json["area_mm2"].sum()

        metricas_comparacion = None
        if info_comparacion:
            metricas_comparacion = self.comparar_detecciones(info_comparacion)

        resumen_callos = df_callos.groupby("imagen").agg({
            "callo_id": "count",
            "area_mm2": ["mean", "std", "min", "max", "sum"]
        }).round(2)
        resumen_callos.columns = [
            "num_callos", "area_media_mm2", "area_std_mm2",
            "area_min_mm2", "area_max_mm2", "area_total_mm2"
        ]

        resumen_semillas = df_semillas.groupby("imagen").agg({
            "callo_id": "count",
            "area_mm2": ["mean", "std", "min", "max", "sum"]
        }).round(2)
        resumen_semillas.columns = [
            "num_semillas", "area_media_mm2", "area_std_mm2",
            "area_min_mm2", "area_max_mm2", "area_total_mm2"
        ]

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        csv_detalle = self.carpeta_salida / "resultados_detallados_{}.csv".format(timestamp)
        df.to_csv(csv_detalle, index=False, encoding="utf-8")
        
        csv_callos = self.carpeta_salida / "resumen_callos_{}.csv".format(timestamp)
        resumen_callos.to_csv(csv_callos, encoding="utf-8")
        
        if not resumen_semillas.empty:
            csv_semillas = self.carpeta_salida / "resumen_semillas_{}.csv".format(timestamp)
            resumen_semillas.to_csv(csv_semillas, encoding="utf-8")
        
        if metricas_comparacion:
            csv_comparacion = self.carpeta_salida / "metricas_comparacion_{}.csv".format(timestamp)
            comparacion_df = pd.DataFrame(metricas_comparacion["metricas_por_imagen"])
            comparacion_df.to_csv(csv_comparacion, index=False, encoding="utf-8")

        print("_" * 50)
        print("\nRESULTADOS GENERALES:")
        print("-" * 30)
        print("Total de objetos detectados: {}".format(total_objetos))
        print("  • Callos: {} objetos".format(total_callos))
        print("  • Semillas: {} objetos".format(total_semillas))
        
        print("\nDETALLE DE CALLOS:")
        print("-" * 25)
        print("Total callos: {}".format(total_callos))
        print("  • Deteccion automatica: {} callos".format(callos_auto))
        print("  • Anotaciones JSON: {} callos".format(callos_json))
        print("Area total de callos: {:.2f} mm²".format(area_callos))
        print("  • Automatico: {:.2f} mm²".format(area_callos_auto))
        print("  • JSON: {:.2f} mm²".format(area_callos_json))
        
        if total_callos > 0:
            print("Area media por callo: {:.2f} mm²".format(df_callos["area_mm2"].mean()))
            print("Circularidad media callos: {:.3f}".format(df_callos["circularidad"].mean()))
        
        print("\nDETALLE DE SEMILLAS:")
        print("-" * 25)
        print("Total semillas: {} (todas del JSON)".format(semillas_json))
        if total_semillas > 0:
            print("Area total de semillas: {:.2f} mm²".format(area_semillas))
            print("Area media por semilla: {:.2f} mm²".format(df_semillas["area_mm2"].mean()))
            print("Circularidad media semillas: {:.3f}".format(df_semillas["circularidad"].mean()))
        
        if metricas_comparacion:
            print("\nCOMPARACION DETECCION vs JSON:")
            print("-" * 30)
            metricas = metricas_comparacion["metricas_globales"]
            print("Verdaderos positivos: {} (callos detectados correctamente)".format(metricas["verdaderos_positivos"]))
            print("Falsos positivos: {} (callos detectados pero no en JSON)".format(metricas["falsos_positivos"]))
            print("Falsos negativos: {} (callos en JSON no detectados)".format(metricas["falsos_negativos"]))
            print("\nMETRICAS DE PRECISION:")
            print("  Precision: {:.1%} (VP / (VP + FP))".format(metricas["precision"]))
            print("  Recall: {:.1%} (VP / (VP + FN))".format(metricas["recall"]))
            print("  F1-Score: {:.1%}".format(metricas["f1"]))
            print("\nNOTA: Estas metricas comparan la deteccion automatica")
            print("      contra las anotaciones manuales del JSON.")
        
        print("\nRESUMEN POR IMAGEN:")
        print("-" * 30)
        
        for imagen in sorted(df["imagen"].unique()):
            df_imagen = df[df["imagen"] == imagen]
            callos_img = df_imagen[df_imagen["tipo"] == "callo"]
            semillas_img = df_imagen[df_imagen["tipo"] == "semilla"]
            
            callos_auto_img = callos_img[callos_img["fuente"] == "Automatico"]
            callos_json_img = callos_img[callos_img["fuente"] == "JSON"]
            
            print("\n{}:".format(imagen))
            print("  Callos: {} total ({} auto, {} JSON)".format(
                len(callos_img), len(callos_auto_img), len(callos_json_img)))
            print("  Semillas: {} (JSON)".format(len(semillas_img)))

        print("\n" + "_" * 50)
        print("ARCHIVOS GUARDADOS:")
        print("-" * 30)
        print("  Resultados detallados: {}".format(csv_detalle.name))
        print("  Resumen callos: {}".format(csv_callos.name))
        if not resumen_semillas.empty:
            print("  Resumen semillas: {}".format(csv_semillas.name))
        if metricas_comparacion:
            print("  Metricas comparacion: {}".format(csv_comparacion.name))
        print("  Visualizaciones: carpeta 'visualizaciones/'")
        print("  Filtros LoG: carpeta 'log_sigma/'")
        print("_" * 50)

        return df, {"callos": resumen_callos, "semillas": resumen_semillas,
                   "comparacion": metricas_comparacion}

In [21]:
def main():
    print("\n" + "_" * 50)
    print("PRACTICA C: DETECCION DE PATRONES Y SEGMENTACION")
    print("ANALISIS DE CALLOS Y SEMILLAS VEGETALES")
    print("_" * 50)

    analizador = AnalizadorCallos(IMAGES_PATH)

    resultados, info_comparacion = analizador.procesar_todas_imagenes(
        usar_alpha=True, 
        usar_log=True,
        usar_json=True
    )

    df_resultados, resumenes = analizador.generar_reporte(resultados, info_comparacion)

    print("\nAnalisis completado exitosamente.\n")


if __name__ == "__main__":
    main()


__________________________________________________
PRACTICA C: DETECCION DE PATRONES Y SEGMENTACION
ANALISIS DE CALLOS Y SEMILLAS VEGETALES
__________________________________________________

PROCESANDO IMAGENES
________________________________________
Total de imagenes: 16
Carpeta: C:\Users\Usuario\Documents\Bioimagenes\Trabajo entregable bioimagen\CallosVegetales
Alpha: Activado
Filtros LoG: Activados
Anotaciones JSON: Activadas



Procesando: 100%|██████████| 16/16 [03:31<00:00, 13.21s/img]



__________________________________________________
PRACTICA: DETECCION DE PATRONES EN CALLOS VEGETALES
__________________________________________________

RESULTADOS GENERALES:
------------------------------
Total de objetos detectados: 211
  • Callos: 183 objetos
  • Semillas: 28 objetos

DETALLE DE CALLOS:
-------------------------
Total callos: 183
  • Deteccion automatica: 65 callos
  • Anotaciones JSON: 118 callos
Area total de callos: 7397.60 mm²
  • Automatico: 854.49 mm²
  • JSON: 6543.12 mm²
Area media por callo: 40.42 mm²
Circularidad media callos: 0.583

DETALLE DE SEMILLAS:
-------------------------
Total semillas: 28 (todas del JSON)
Area total de semillas: 8.61 mm²
Area media por semilla: 0.31 mm²
Circularidad media semillas: 0.806

COMPARACION DETECCION vs JSON:
------------------------------
Verdaderos positivos: 19 (callos detectados correctamente)
Falsos positivos: 46 (callos detectados pero no en JSON)
Falsos negativos: 99 (callos en JSON no detectados)

METRICAS DE